In [16]:
# ==============================================================================
# 第一步：环境设置与库导入
# ==============================================================================
import pandas as pd
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt
import matplotlib
from sklearn.preprocessing import StandardScaler
from sklearn.covariance import GraphicalLassoCV
from scipy.stats import gmean
from pathlib import Path
import warnings

# 忽略将来可能出现的警告，保持输出整洁
warnings.filterwarnings('ignore', category=FutureWarning)

# --- 绘图风格与中文支持设置 ---
# 设置全局字体，确保中文和负号正常显示
matplotlib.rcParams['font.family'] = 'SimHei'  # 使用 'SimHei' 字体
matplotlib.rcParams['axes.unicode_minus'] = False  # 解决负号显示问题
matplotlib.rcParams['font.size'] = 16  # 设置稍大的全局字号
matplotlib.rcParams['axes.labelsize'] = 18 # 坐标轴标签字号
matplotlib.rcParams['xtick.labelsize'] = 14 # x轴刻度字号
matplotlib.rcParams['ytick.labelsize'] = 14 # y轴刻度字号

print("第一步：库导入与环境设置完成。")

第一步：库导入与环境设置完成。


In [17]:
# ==============================================================================
# 第二步：文件路径与输出目录配置
# ==============================================================================
# 输入文件路径 (相对于当前脚本)
# 假设您的数据处理脚本在 '../DataPreprocessing/'，输出在'../Data/'
# 那么此脚本可以放在 '../Problem4/'，路径正确
input_file = Path('../../Data/0/处理后的数据.xlsx')

# 结果输出目录 (在当前脚本所在文件夹下创建'Result'目录)
output_dir = Path('./Result')
# 确保输出目录存在，如果不存在则创建
output_dir.mkdir(parents=True, exist_ok=True)

print(f"第二步：文件路径配置完成。")
print(f"    - 输入文件: {input_file}")
print(f"    - 输出目录: {output_dir}")

第二步：文件路径配置完成。
    - 输入文件: ..\..\Data\0\处理后的数据.xlsx
    - 输出目录: Result


In [18]:
# ==============================================================================
# 第三步：数据加载与准备
# ==============================================================================
try:
    df_classified = pd.read_excel(input_file, sheet_name='已分类清洗后数据')
    print("第三步：成功加载 '已分类清洗后数据'。")
except FileNotFoundError:
    print(f"错误：输入文件不存在于路径 -> {input_file}")
    print("请检查文件路径配置是否正确。")
    # 如果文件不存在，退出程序
    exit()

# 识别化学成分列 (排除所有非数值和ID列)
chemical_columns = [
    col for col in df_classified.columns 
    if df_classified[col].dtype in ['float64', 'int64'] 
    and col not in ['文物编号', '成分总和']
]
print(f"    - 识别出 {len(chemical_columns)} 个化学成分用于分析。")

第三步：成功加载 '已分类清洗后数据'。
    - 识别出 14 个化学成分用于分析。


In [19]:
# ==============================================================================
# 第四步：核心分析与可视化（清亮柔彩版）
# ==============================================================================
from matplotlib.path import Path
import matplotlib.patches as patches
import matplotlib.lines as mlines
import networkx.algorithms.community as nx_comm # 导入社群发现模块

# --- 辅助函数：绘制“两端宽、中间细”的连接 ---
def draw_pinched_edge(ax, pos1, pos2, width=1.0, color='blue', alpha=0.7):
    """
    参数经过精心微调，实现更平滑的“收腰”效果。
    """
    style = patches.ArrowStyle.Wedge(tail_width=width * 2.5, shrink_factor=0.25)
    p1, p2 = np.array(pos1), np.array(pos2)
    mid_point = (p1 + p2) / 2
    arrow1 = patches.FancyArrowPatch(p1, mid_point, arrowstyle=style, color=color, alpha=alpha, shrinkA=15, shrinkB=0, mutation_scale=25, edgecolor=None)
    arrow2 = patches.FancyArrowPatch(p2, mid_point, arrowstyle=style, color=color, alpha=alpha, shrinkA=15, shrinkB=0, mutation_scale=25, edgecolor=None)
    ax.add_patch(arrow1)
    ax.add_patch(arrow2)

# --- 主分析与可视化函数 ---
def analyze_and_plot_glasso_network(df, glass_type, chemical_cols, output_path):
    """
    主分析函数，集成了最终的“清亮柔彩”色彩方案。
    """
    # 预处理化学成分列名，去除中文括号及内部内容，只保留英文缩写
    processed_chemical_cols = []
    for col in chemical_cols:
        if '(' in col and ')' in col:
            processed_chemical_cols.append(col.split('(')[1][:-1]) 
        else:
            processed_chemical_cols.append(col)

    data = df[df['类型'] == glass_type][chemical_cols].copy()
    data = data.loc[:, (data != 0).any(axis=0)]
    actual_chemical_cols_for_analysis = data.columns.tolist() 
    original_to_english_label_map = {original_col: processed_chemical_cols[chemical_cols.index(original_col)] for original_col in actual_chemical_cols_for_analysis}

    print(f"\n开始分析 {glass_type} 玻璃 ({len(data)}个样本)...")
    delta = 1e-6
    data[data <= 0] = delta
    geom_mean = gmean(data, axis=1)
    data_clr = np.log(data.div(geom_mean, axis=0))
    scaler = StandardScaler()
    data_clr_scaled = scaler.fit_transform(data_clr)
    model = GraphicalLassoCV(cv=5, assume_centered=True, max_iter=200, n_jobs=-1)
    model.fit(data_clr_scaled)
    precision_matrix = model.precision_
    print(f"    - 找到最优 Alpha (正则化强度): {model.alpha_:.4f}")
    D = precision_matrix.shape[0]
    partial_corr = np.zeros_like(precision_matrix)
    for i in range(D):
        for j in range(D):
            if i != j:
                partial_corr[i, j] = -precision_matrix[i, j] / np.sqrt(precision_matrix[i, i] * precision_matrix[j, j])
    np.fill_diagonal(partial_corr, 1)
    partial_corr_df = pd.DataFrame(partial_corr, index=actual_chemical_cols_for_analysis, columns=actual_chemical_cols_for_analysis)
    output_excel_path = output_path / f"{glass_type}玻璃_偏相关系数矩阵.xlsx"
    partial_corr_df.to_excel(output_excel_path)
    print(f"    - 已保存偏相关系数矩阵 -> {output_excel_path}")

    # --- 7. 网络可视化（清亮柔彩版） ---
    G = nx.Graph()
    for col in actual_chemical_cols_for_analysis: 
        G.add_node(col, label=original_to_english_label_map[col])

    threshold = 0.01
    for i in range(D):
        for j in range(i + 1, D):
            p_corr = partial_corr[i, j]
            if abs(p_corr) > threshold:
                G.add_edge(actual_chemical_cols_for_analysis[i], actual_chemical_cols_for_analysis[j], weight=abs(p_corr), pcorr=p_corr)

    # --- 核心升级：采用“清亮柔彩”色板 ---
    LIGHT_BRIGHT_PALETTE = [
        '#FF6B6B',  # 柔红
        '#4ECDC4',  # 薄荷绿
        '#45B7D1',  # 天蓝
        '#FED766',  # 柔黄
        '#7E78D2',  # 柔紫
        '#FF9F1C',  # 柔橙
        '#C1E79A',  # 豆绿
    ]
    
    communities = nx_comm.louvain_communities(G, weight='weight', seed=42)
    num_communities = len(communities)
    print(f"    - 发现 {num_communities} 个结构社群。")

    node_to_community = {}
    for i, community in enumerate(communities):
        for node in community:
            node_to_community[node] = i
    
    node_colors = [LIGHT_BRIGHT_PALETTE[node_to_community[node] % len(LIGHT_BRIGHT_PALETTE)] for node in G.nodes()]
    
    unweighted_degree = dict(G.degree())
    weighted_degree = dict(G.degree(weight='weight'))
    scores = {node: unweighted_degree[node] * weighted_degree[node] for node in G.nodes()}
    max_score = max(scores.values()) if scores else 1
    normalized_scores = {node: score / max_score for node, score in scores.items()}
    node_sizes = [normalized_scores[node] * 12000 + 4000 for node in G.nodes()]

    fig, ax = plt.subplots(figsize=(22, 22))
    pos = nx.spring_layout(G, k=1.8, iterations=100, seed=42)

    # A. 绘制边（使用新的橙绿对比色）
    edge_colors_map = {'positive': '#2A9D8F', 'negative': '#F4A261'} # 饱和度稍低的 绿 / 橙
    for u, v in G.edges():
        pcorr = G.edges[u, v]['pcorr']
        color = edge_colors_map['positive' if pcorr > 0 else 'negative']
        width = abs(pcorr) * 1.5 + 0.1
        draw_pinched_edge(ax, pos[u], pos[v], width=width, color=color, alpha=0.75)

    # B. 绘制节点和标签
    node_labels = {node: G.nodes[node]['label'] for node in G.nodes()}
    nx.draw_networkx_nodes(G, pos, ax=ax, nodelist=G.nodes(), node_size=node_sizes, node_color=node_colors, alpha=1, edgecolors='black', linewidths=2.5)
    nx.draw_networkx_labels(G, pos, ax=ax, labels=node_labels, font_size=24, font_family='Arial', font_weight='bold', font_color='black')

    # C. 添加黑色边框
    for spine in ax.spines.values():
        spine.set_visible(True)
        spine.set_color('black')
        spine.set_linewidth(3)
    ax.tick_params(left=False, bottom=False, labelleft=False, labelbottom=False)

    # D. 创建动态图例（中文）
    legend_handles = []
    for i in range(num_communities):
        legend_handles.append(
            mlines.Line2D([], [], color=LIGHT_BRIGHT_PALETTE[i % len(LIGHT_BRIGHT_PALETTE)], marker='o', linestyle='None',
                          markersize=25, markeredgecolor='black', label=f'结构社群 {i+1}')
        )
    legend_handles.append(
        mlines.Line2D([], [], color=edge_colors_map['positive'], marker='s', linestyle='None',
                      markersize=18, label='正相关 (协同)')
    )
    legend_handles.append(
        mlines.Line2D([], [], color=edge_colors_map['negative'], marker='s', linestyle='None',
                      markersize=18, label='负相关 (拮抗)')
    )

    ax.legend(handles=legend_handles, 
              loc='upper right', 
              fontsize=22,
              frameon=True,
              edgecolor='black',
              facecolor='white',
              framealpha=0.9)

    ax.set_aspect('equal')
    plt.tight_layout()
    
    output_png_path = output_path / f"{glass_type}_Network_Final_Soft_Bright.png" # 更新文件名
    plt.savefig(output_png_path, dpi=300, bbox_inches='tight')
    print(f"    - 已保存最终清亮柔彩版的网络图 -> {output_png_path}")
    plt.close(fig)

print("第四步：清亮柔彩版核心分析函数定义完成。")

第四步：清亮柔彩版核心分析函数定义完成。


In [2]:
# ==============================================================================
# 第五步：执行分析流程
# ==============================================================================
if __name__ == "__main__":
    print("\n==================== 开始执行分析流程 ====================")
    
    # 对高钾玻璃进行分析
    analyze_and_plot_glasso_network(
        df=df_classified, 
        glass_type='高钾', 
        chemical_cols=chemical_columns,
        output_path=output_dir
    )
    
    # 对铅钡玻璃进行分析
    analyze_and_plot_glasso_network(
        df=df_classified, 
        glass_type='铅钡', 
        chemical_cols=chemical_columns,
        output_path=output_dir
    )
    
    print("\n==================== 分析流程全部执行完毕 ====================")


==================== 开始执行分析流程 ====================


NameError: name 'analyze_and_plot_glasso_network' is not defined